In [8]:
import pandas as pd
import datetime
import sys, os
import numpy as np
from os import path
import datetime as dt
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.svm import SVC
import itertools
import xgboost as xgb
import seaborn as sns
from typing import Union, Dict, List
from pybit.unified_trading import HTTP

sys.path.insert(0, os.path.join(os.getcwd(), ".."))
import fin_utilities 
import matplotlib.pyplot as plt
cfg = fin_utilities.__cfg_reading("pred")


#td = TDClient(apikey=cfg['TWELVEDATA']['API'])  

PROJECT_DIR = eval(cfg['PROJECT_PATH'])
DATA_PATH   = PROJECT_DIR / cfg['DATA_FOLDER']
SOURCE      = cfg['API_DATA_SOURCE']
URL         = cfg[SOURCE]['API_URL_HIST_DATA']
COLUMNS     = cfg[SOURCE]['COLUMN_NAMES']
PARAMS      = cfg[SOURCE]['REQ_PARAMS']
SYMBOL      = cfg['SYMBOL']
STABLECOIN  = cfg['STABLECOIN']
PAIR        = SYMBOL + STABLECOIN #binance
MAX_LENGTH  = cfg['MAX_TRADE_DURATION']
XGB_PARAM   = cfg['xgb']

BYBIT_API_KEY = "5u0HfwB5UPJeiQo3WR"
BYBIT_SECRET_KEY = "hjFn5aEvyuVEZ1dnna6R4s1NS1vw3vZdJFIL"


session = HTTP(
    testnet=False,
    api_key=BYBIT_API_KEY,
    api_secret=BYBIT_SECRET_KEY,
)

In [3]:
# COPPIE DISPONIBILI
result = session.get_tickers(
    category="linear"
).get('result')['list']

tickers = [asset['symbol'] for asset in result if asset['symbol'].endswith('USDT')]
print(tickers)

['10000000AIDOGEUSDT', '1000000BABYDOGEUSDT', '1000000MOGUSDT', '1000000PEIPEIUSDT', '10000COQUSDT', '10000LADYSUSDT', '10000NFTUSDT', '10000SATSUSDT', '10000STARLUSDT', '10000WENUSDT', '1000APUUSDT', '1000BEERUSDT', '1000BONKUSDT', '1000BTTUSDT', '1000FLOKIUSDT', '1000LUNCUSDT', '1000PEPEUSDT', '1000RATSUSDT', '1000TURBOUSDT', '1000XECUSDT', '1CATUSDT', '1INCHUSDT', 'AAVEUSDT', 'ACEUSDT', 'ACHUSDT', 'ADAUSDT', 'AERGOUSDT', 'AEVOUSDT', 'AGIUSDT', 'AGLDUSDT', 'AIOZUSDT', 'AIUSDT', 'AKROUSDT', 'AKTUSDT', 'ALGOUSDT', 'ALICEUSDT', 'ALPACAUSDT', 'ALPHAUSDT', 'ALTUSDT', 'AMBUSDT', 'ANKRUSDT', 'ANTUSDT', 'APEUSDT', 'API3USDT', 'APTUSDT', 'ARBUSDT', 'ARKMUSDT', 'ARKUSDT', 'ARPAUSDT', 'ARUSDT', 'ASTRUSDT', 'ATAUSDT', 'ATHUSDT', 'ATOMUSDT', 'AUCTIONUSDT', 'AUDIOUSDT', 'AVAXUSDT', 'AXLUSDT', 'AXSUSDT', 'BADGERUSDT', 'BAKEUSDT', 'BALUSDT', 'BANDUSDT', 'BATUSDT', 'BBUSDT', 'BCHUSDT', 'BEAMUSDT', 'BELUSDT', 'BENDOGUSDT', 'BICOUSDT', 'BIGTIMEUSDT', 'BLASTUSDT', 'BLURUSDT', 'BLZUSDT', 'BNBUSDT', 'BNTU

In [24]:

response = session.get_kline(category='linear', 
                             symbol='ETHUSDT', 
                             interval='D').get('result')

def format_data(response):    
    data = response.get('list', None)
    if not data:
        return 
    
    data = pd.DataFrame(data,
                        columns =[
                            'timestamp',
                            'open',
                            'high',
                            'low',
                            'close',
                            'volume',
                            'turnover'
                            ],
                        )
    data = data[::-1].apply(pd.to_numeric)
    data['date'] = pd.to_datetime(data['timestamp']/1000,unit='s')
    return data

df = format_data(response)
df

,timestamp,open,high,low,close,volume,turnover,date
199,1703289600000,2327.02,2332.99,2266.05,2310.40,511643.02,1.173304e+09,2023-12-23
198,1703376000000,2310.40,2329.14,2242.17,2265.61,511477.49,1.170031e+09,2023-12-24
197,1703462400000,2265.61,2307.97,2254.81,2272.80,422808.90,9.640830e+08,2023-12-25
196,1703548800000,2272.80,2277.20,2180.00,2232.79,673992.19,1.503258e+09,2023-12-26
195,1703635200000,2232.79,2396.30,2213.16,2380.01,974204.09,2.261616e+09,2023-12-27
...,...,...,...,...,...,...,...,...
4,1720137600000,3058.64,3108.34,2802.87,2980.00,1976899.82,5.804147e+09,2024-07-05
3,1720224000000,2980.00,3081.46,2952.10,3065.39,531386.25,1.602516e+09,2024-07-06
2,1720310400000,3065.39,3071.95,2918.03,2929.87,543091.78,1.624483e+09,2024-07-07
1,1720396800000,2929.87,3096.00,2820.00,3017.91,1571584.45,4.657722e+09,2024-07-08


In [32]:
import time 
def get_last_timestamp(df):
    return int(df.timestamp[-1:].values[0])

start = int(dt.datetime(2023, 1, 1).timestamp()* 1000)

interval = 60
symbol = 'DOGEUSDT'

def get_data_from(
    category: str,
    symbol: str,
    start: str,
    end: str,
    interval: int,
):
    
    df = pd.DataFrame()
    while True:
        response = session.get_kline(category=category, 
                                    symbol=symbol, 
                                    start=start,
                                    interval=interval).get('result')
        
        latest = format_data(response)
        print(len(latest))
        if end is not None and latest > end:
            return df[df['timestamp']<=end]
        
        if not isinstance(latest, pd.DataFrame):
            break
        
        start = get_last_timestamp(latest)
        time.sleep(0.1)
        
        df = pd.concat([df, latest])
        print(f'Collecting data starting {dt.datetime.fromtimestamp(start/1000)}')
        if len(latest) == 1: break

    df.drop_duplicates(subset=['timestamp'], keep='last', inplace=True)

    return df


doge = get_data_from(
    category='linear',
    symbol=symbol,
    start=start,
    end=None,
    interval=60,
)

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
9
1


In [73]:
def generate_labels(data, lookahead=10, tp=0.03, sl=0.015):
    labels = []
    for i in range(len(data) - lookahead):
        current_price = data['close'].iloc[i]
        future_prices = data['close'].iloc[i+1:i+1+lookahead]
        
        tp_reached = False
        for future_price in future_prices:
            if (future_price - current_price) / current_price >= tp:
                tp_reached = True
                labels.append(1)
                break
            if (current_price - future_price) / current_price >= sl:
                labels.append(0)
                break
                
        if not tp_reached and len(labels) == i:
            labels.append(0)  # Append 0 if neither TP nor SL was reached

    # Pad labels to match the length of the DataFrame
    labels += [0] * lookahead  # Fill the remaining labels with 0
    return labels

def return_index_if_exists(df_series, curr_idx, val, pos_crit, max_length):

    current_val = val
    if pos_crit:
        try:
            thing_index = list(el >= current_val for el in df_series[curr_idx+1:]).index(True) +1
        except ValueError:
            thing_index = max_length
    else:
        try:   
            thing_index = list(el <= current_val for el in df_series[curr_idx+1:]).index(True) +1
        except ValueError:
            thing_index = max_length

    return thing_index

def labelize_output_according_criterion2(df: pd.DataFrame, map_ohlc:dict, wrt:str='close', target_var:str='signal', threshold:float=.015, risk_reward_ratio:float=.5, max_trade_length:int=5) -> pd.DataFrame:
    
    df_ = df.copy()
    positive_criterion = 1 + threshold/risk_reward_ratio
    negative_criterion = 1 - threshold
    
    min_above = []
    min_below = []

    df_['TP'] = df_[map_ohlc[wrt]]*positive_criterion
    df_['SL'] = df_[map_ohlc[wrt]]*negative_criterion

    for idx,row in df_.iterrows():

        pos_val = positive_criterion*row[map_ohlc[wrt]]
        neg_val = negative_criterion*row[map_ohlc[wrt]]

        if idx != df_.index[-1]:
            candidates_above_minima = []
            candidates_below_minima = []
            for _,v in map_ohlc.items():
                candidates_above_minima.append(return_index_if_exists(df_[v], idx, pos_val, True, max_trade_length))
                candidates_below_minima.append(return_index_if_exists(df_[v], idx, neg_val, False, max_trade_length))

            min_above.append(min(candidates_above_minima))
            min_below.append(min(candidates_below_minima))

    min_above.append(None)
    min_below.append(None)

    df_['min_above'] = min_above
    df_['min_below'] = min_below

    df_[target_var] = (df_['min_above'].lt(df_['min_below'])) & (df_['min_above'] <= max_trade_length)
    df_[target_var]=df_[target_var].astype(int)

    df_ = df_.drop(columns=['min_above','min_below'])

    return df_


doge = labelize_output_according_criterion2(doge.reset_index(drop=True),map_ohlc = {'open':'open','high':'high','low':'low','close':'close'})
doge['lavel'] = generate_labels(doge)
doge.to_csv("data/data.csv", index=False)

In [75]:
doge

,timestamp,open,high,low,close,volume,turnover,date,label,lavel,TP,SL,signal
0,1672527600000,0.07019,0.07034,0.06989,0.07021,25993134,1.822307e+06,2022-12-31 23:00:00,0,0,0.072316,0.069157,0
1,1672531200000,0.07021,0.07031,0.06933,0.06979,83742628,5.839573e+06,2023-01-01 00:00:00,-1,0,0.071884,0.068743,0
2,1672534800000,0.06979,0.07003,0.06949,0.06991,26575163,1.852284e+06,2023-01-01 01:00:00,0,0,0.072007,0.068861,0
3,1672538400000,0.06991,0.07037,0.06976,0.06983,40950401,2.869860e+06,2023-01-01 02:00:00,-1,0,0.071925,0.068783,0
4,1672542000000,0.06983,0.06991,0.06956,0.06983,22241756,1.551288e+06,2023-01-01 03:00:00,-1,0,0.071925,0.068783,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13337,1720540800000,0.10775,0.10844,0.10723,0.10777,40373320,4.351702e+06,2024-07-09 16:00:00,-1,0,0.111003,0.106153,0
13338,1720544400000,0.10777,0.10808,0.10661,0.10685,34083635,3.655076e+06,2024-07-09 17:00:00,-1,0,0.110056,0.105247,0
13339,1720548000000,0.10685,0.10788,0.10660,0.10737,37486476,4.027832e+06,2024-07-09 18:00:00,-1,0,0.110591,0.105759,0
13340,1720551600000,0.10737,0.10743,0.10695,0.10734,12686720,1.359661e+06,2024-07-09 19:00:00,-1,0,0.110560,0.105730,0
